# Tests

In [1]:
import pyhelayers
import utils 
import copy
utils.verify_memory()
# Print the ciphertext content for demo purposes
pyhelayers.get_print_options().print_encrypted_content=True 
print("Imported pyhelayers", pyhelayers.VERSION)

Imported pyhelayers 1.3.0


In [2]:
num_slots = 8192 # Number of slots per ciphertext
multiplication_depth = 2 # Allow 2 levels of multiplications
fractional_part_precision = 39  # Set the precision to 1/2^40.
                                #  -> change to 39
integer_part_precision = 21 # Set the largest number to 2^20 -> change to 14, because the highest number is 100^2 < 2^14
                            #  -> change to 20 again, because integer_part seems to be smth different (maybe the polynomes?)
                            #  -> change to 21
                            # SUMMARY: 39,21 seems to be the combination that yields the lowest average deviation
security_level = 128

requirement = pyhelayers.HeConfigRequirement(
    num_slots,
    multiplication_depth,
    fractional_part_precision,
    integer_part_precision,
    security_level)

he_context = pyhelayers.DefaultContext()
he_context.init(requirement)

In [3]:
# Create the Encoder using the context.
encoder = pyhelayers.Encoder(he_context)

In [4]:
class ResStorage:
    execTimeTotal = 0
    deviationTotal = 0

import time
import random

num_execs = 1000

ccSum = ResStorage()
ccSub = ResStorage()
ccMul = ResStorage()
cSq   = ResStorage()
cNeg  = ResStorage()
cPow  = ResStorage()
cRotR = ResStorage()
cRotL = ResStorage()
cpSum = ResStorage()
cpSub = ResStorage()
cpMul = ResStorage()

def setup():
    global v1, p1, c1, v2, p2, c2
    v1 = [0] * num_slots
    v2 = [0] * num_slots
    for i in range(num_slots):
        v1[i] = random.randint(1,100)
        #v1[i] = 100
    p1 = encoder.encode(v1)
    c1 = encoder.encode_encrypt(v1)

    for i in range(num_slots):
        v2[i] = random.randint(1,100)
        #v2[i] = 100
    p2 = encoder.encode(v2)
    c2 = encoder.encode_encrypt(v2)
    
# Multiplication
def fun_ccMul():
    setup()
    start = time.time()
    
    c1.multiply(c2)
    c1.multiply(c2)
    
    end = time.time()
    ccMul.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        ccMul.deviationTotal += abs(res[i] - actualRes)
    
# Addition
def fun_ccSum():
    setup()
    start = time.time()
    
    c1.add(c2)
    c1.add(c2)
    
    end = time.time()
    ccSum.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        ccSum.deviationTotal += abs(res[i] - actualRes)
    
# Subtraction
def fun_ccSub():
    setup()
    start = time.time()
    
    c1.sub(c2)
    c1.sub(c2)
    
    end = time.time()
    ccSub.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Squaring
def fun_cSq():
    setup()
    start = time.time()
    
    c1.square()
    
    end = time.time()
    cSq.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = pow(v1[i],2)
        cSq.deviationTotal += abs(res[i] - actualRes)
    
# Negation
def fun_cNeg():
    setup()
    start = time.time()
    
    c1.negate()
    c1.negate()
    
    end = time.time()
    cNeg.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i]
        cNeg.deviationTotal += abs(res[i] - actualRes)
    
# Left Rotation
def fun_cRotL():
    setup()
    start = time.time()
    
    c1.rotate(1)
    c1.rotate(1)
    
    end = time.time()
    cRotL.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = 0
        if (i + 2 >= num_slots):
            actualRes = v1[i + 2 - num_slots]
        else:
            actualRes = v1[i + 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    
# Right Rotation
def fun_cRotR():
    setup()
    start = time.time()
    
    c1.rotate(-1)
    c1.rotate(-1)
    
    end = time.time()
    cRotR.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = 0
        if (i - 2 < 0):
            actualRes = v1[num_slots + (i - 2)]
        else:
            actualRes = v1[i - 2]
        cRotR.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Addition
def fun_cpSum():
    setup()
    start = time.time()
    
    c1.add_plain(p2)
    c1.add_plain(p2)
    
    end = time.time()
    cpSum.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        cpSum.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Subtraction
def fun_cpSub():
    setup()
    start = time.time()
    
    c1.sub_plain(p2)
    c1.sub_plain(p2)
    
    end = time.time()
    cpSub.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] - v2[i] - v2[i]
        cpSub.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Multiplication
def fun_cpMul():
    setup()
    start = time.time()
    
    c1.multiply_plain(p2)
    c1.multiply_plain(p2)
    
    end = time.time()
    cpMul.execTimeTotal += end - start
    
    res = encoder.decrypt_decode_double(c1)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        cpMul.deviationTotal += abs(res[i] - actualRes)
    
if __name__ == "__main__":
    
    for i in range(num_execs):
        fun_ccSum()
    for i in range(num_execs):
        fun_ccSub()
    for i in range(num_execs):
        fun_ccMul()
    for i in range(num_execs):
        fun_cSq()
    for i in range(num_execs):
        fun_cNeg()
    for i in range(num_execs):
        fun_cRotL()
    for i in range(num_execs):
        fun_cRotR()
    for i in range(num_execs):
        fun_cpSum()
    for i in range(num_execs):
        fun_cpSub()
    for i in range(num_execs):
        fun_cpMul()

In [5]:
    print('Execution time in milliseconds:')
    print()
    print('Addition:')    # Formula: Total time or deviation for all cycles / operations per cycle
    print('Average execution time: {:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
    print()
    print('Subtraction:')
    print('Average execution time: {:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
    print()
    print('Multiplication:')
    print('Average execution time: {:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 )) # KOMMENTIEREN
    print('Average deviation:      {:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
    print()
    print('Squaring:')    # Square only has 1 operation per cycle, otherwise the configuration needs to be changed
    print('Average execution time: {:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 )) # ALLE SQUARES ÜBERPRÜFEN
    print('Average deviation:      {:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
    print()
    print('Negation:')
    print('Average execution time: {:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
    print()
    print('Left Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
    print()
    print('Right Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Addition:')
    print('Average execution time: {:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Subtraction:')
    print('Average execution time: {:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Multiplication:')
    print('Average execution time: {:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))
    
    PRINT_RAW_NUMBERS = True
    if (PRINT_RAW_NUMBERS):
        print()
        print()
        print('Raw numbers:')
        print()
        print('Average execution time:')
        print('{:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
        print('{:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print()
        print('Average deviation:')
        print('{:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
        print('{:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))

Execution time in milliseconds:

Addition:
Average execution time: 0.128108
Average deviation:      0.000032

Subtraction:
Average execution time: 0.165391
Average deviation:      0.000032

Multiplication:
Average execution time: 11.093197
Average deviation:      0.099932

Squaring:
Average execution time: 12.986287
Average deviation:      0.002897

Negation:
Average execution time: 0.144397
Average deviation:      0.000014

Left Rotation:
Average execution time: 9.512270
Average deviation:      0.000096

Right Rotation:
Average execution time: 9.776214
Average deviation:      0.000096

Ciphertext-Plain Addition:
Average execution time: 0.110627
Average deviation:      0.000014

Ciphertext-Plain Subtraction:
Average execution time: 0.112835
Average deviation:      0.000014

Ciphertext-Plain Multiplication:
Average execution time: 5.158503
Average deviation:      0.048579


Raw numbers:

Average execution time:
0.128108
0.165391
11.093197
12.986287
0.144397
9.512270
9.776214
0.110627
0.